<a href="https://colab.research.google.com/github/AG-2002/Customer-Segmentation-and-Prediction-of-Term-Deposit-Subscriptions-in-Bank-Marketing/blob/main/Bank_Data(Model_fitting(DecisionTree%2CRandomForest)).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('bank_data_cleaned.csv')
df

,age,month,day_of_week,duration,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,poutcome_nonexistent,poutcome_success,pdays_new_recently contacted,pdays_new_sometimes ago,y
0,56,5,1,261,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
1,57,5,1,149,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
2,37,5,1,226,1,0,1.1,93.994,-36.4,4.857,...,0,1,0,0,1,1,0,0,0,0
3,40,5,1,151,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
4,56,5,1,307,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41171,73,11,5,334,1,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,1
41172,46,11,5,383,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,0
41173,56,11,5,189,2,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,0
41174,44,11,5,442,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,1


# **----------  DECISION TREE ----------**

In [40]:
DT=DecisionTreeClassifier(random_state=42)
DT.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [41]:
# Predict and evaluate
y_pred = DT.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.8919378338999514
[[6865  443]
 [ 447  481]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      7308
           1       0.52      0.52      0.52       928

    accuracy                           0.89      8236
   macro avg       0.73      0.73      0.73      8236
weighted avg       0.89      0.89      0.89      8236



In [5]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.8 MB/s eta 0:00:00


In [23]:
import optuna
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

In [7]:
X = df.drop(columns='y')
y = df['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
def objective(trial):
    # Suggest hyperparameters
    params = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "splitter": trial.suggest_categorical("splitter", ["best", "random"]),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    }

    model = DecisionTreeClassifier(**params, random_state=42)

    # Use cross-validation to evaluate
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy").mean()
    return score

In [9]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Show best parameters
print("Best hyperparameters:")
print(study.best_params)

[I 2025-07-28 15:56:31,471] A new study created in memory with name: no-name-0c032fde-1f2b-4797-b2ca-ad8b85dc0d34
[I 2025-07-28 15:56:31,613] Trial 0 finished with value: 0.8982695810564664 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 9, 'min_samples_split': 15, 'min_samples_leaf': 11, 'max_features': 'log2'}. Best is trial 0 with value: 0.8982695810564664.
[I 2025-07-28 15:56:31,770] Trial 1 finished with value: 0.8948087431693988 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 38, 'min_samples_split': 10, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8982695810564664.
[I 2025-07-28 15:56:32,209] Trial 2 finished with value: 0.9034001214329083 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 33, 'min_samples_split': 5, 'min_samples_leaf': 12, 'max_features': None}. Best is trial 2 with value: 0.9034001214329083.
[I 2025-07-28 15:56:32,758] Trial 3 finished with value: 0.

Best hyperparameters:
{'criterion': 'gini', 'splitter': 'best', 'max_depth': 48, 'min_samples_split': 16, 'min_samples_leaf': 18, 'max_features': None}


In [10]:
# Train final model
dt = DecisionTreeClassifier(**study.best_params, random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=48, min_samples_leaf=18, min_samples_split=16,
                       random_state=42)

In [11]:
# Predict and evaluate
y_pred = dt.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.9072365225837785
[[6997  311]
 [ 453  475]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      7308
           1       0.60      0.51      0.55       928

    accuracy                           0.91      8236
   macro avg       0.77      0.73      0.75      8236
weighted avg       0.90      0.91      0.90      8236



In [12]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, recall_score

def objective(trial):
    # Same hyperparameter tuning space as before...
    params = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "splitter": trial.suggest_categorical("splitter", ["best", "random"]),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced", None])
    }

    model = DecisionTreeClassifier(**params, random_state=42)

    # Use custom scoring focused on recall
    recall = cross_val_score(
        model,
        X_train,
        y_train,
        scoring=make_scorer(recall_score, pos_label=1),
        cv=StratifiedKFold(n_splits=5),
    ).mean()

    return recall

In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best Params to Maximize Recall:")
print(study.best_params)


[I 2025-07-28 15:59:52,460] A new study created in memory with name: no-name-c96dbee8-19ee-4ac1-a6bc-efffd6f6fc92
[I 2025-07-28 15:59:52,765] Trial 0 finished with value: 0.7695834980936177 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 33, 'min_samples_split': 12, 'min_samples_leaf': 19, 'max_features': 'log2', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7695834980936177.
[I 2025-07-28 15:59:53,056] Trial 1 finished with value: 0.4349261571613587 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 27, 'min_samples_split': 18, 'min_samples_leaf': 2, 'max_features': 'log2', 'class_weight': None}. Best is trial 0 with value: 0.7695834980936177.
[I 2025-07-28 15:59:53,827] Trial 2 finished with value: 0.5219671108241158 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 49, 'min_samples_split': 3, 'min_samples_leaf': 13, 'max_features': None, 'class_weight': None}. Best is trial 0 with value: 0.769583498093

Best Params to Maximize Recall:
{'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 17, 'min_samples_split': 15, 'min_samples_leaf': 20, 'max_features': None, 'class_weight': 'balanced'}


In [14]:
# Train final model
dt1 = DecisionTreeClassifier(**study.best_params, random_state=42)
dt1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='log_loss',
                       max_depth=17, min_samples_leaf=20, min_samples_split=15,
                       random_state=42, splitter='random')

In [15]:
# Predict and evaluate
y_pred = dt1.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.8533268576979116
[[6171 1137]
 [  71  857]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.84      0.91      7308
           1       0.43      0.92      0.59       928

    accuracy                           0.85      8236
   macro avg       0.71      0.88      0.75      8236
weighted avg       0.93      0.85      0.87      8236



# **-----------  RANDOM FOREST  ----------**

In [38]:
rf=BalancedRandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=42)

In [39]:
# Predict and evaluate
y_pred = rf.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.878581835842642
[[6387  921]
 [  79  849]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.87      0.93      7308
           1       0.48      0.91      0.63       928

    accuracy                           0.88      8236
   macro avg       0.73      0.89      0.78      8236
weighted avg       0.93      0.88      0.89      8236



In [31]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 32),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced", None])
    }

    model = BalancedRandomForestClassifier(**params, random_state=42, n_jobs=-1)

    # 5-fold CV using accuracy as score (you can also use recall, f1, etc.)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="recall_macro").mean()
    return score

In [32]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print best parameters
print("Best hyperparameters:")
print(study.best_params)

[I 2025-07-28 19:56:47,024] A new study created in memory with name: no-name-ab87cf92-1e48-4270-9182-66b136b9f35a
[I 2025-07-28 19:57:12,285] Trial 0 finished with value: 0.829246282335955 and parameters: {'n_estimators': 448, 'max_depth': 15, 'min_samples_split': 7, 'min_samples_leaf': 13, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'log_loss', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.829246282335955.
[I 2025-07-28 19:57:36,043] Trial 1 finished with value: 0.6407872235329031 and parameters: {'n_estimators': 370, 'max_depth': 2, 'min_samples_split': 10, 'min_samples_leaf': 19, 'max_features': None, 'bootstrap': False, 'criterion': 'gini', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.829246282335955.
[I 2025-07-28 19:58:15,598] Trial 2 finished with value: 0.7820090689107052 and parameters: {'n_estimators': 932, 'max_depth': 4, 'min_samples_split': 6, 'min_samples_leaf': 20, 'max_features': 'log2', 'bootstrap': True, 'criterion': 'entropy', 

Best hyperparameters:
{'n_estimators': 507, 'max_depth': 28, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': True, 'criterion': 'entropy', 'class_weight': None}


In [35]:
# Train final model
rf1 = BalancedRandomForestClassifier(**study.best_params, random_state=42)
rf1.fit(X_train, y_train)

BalancedRandomForestClassifier(bootstrap=True, criterion='entropy',
                               max_depth=28, max_features=None,
                               min_samples_split=20, n_estimators=507,
                               random_state=42)

In [36]:
# Predict and evaluate
y_pred = rf1.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.86534725594949
[[6263 1045]
 [  64  864]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.86      0.92      7308
           1       0.45      0.93      0.61       928

    accuracy                           0.87      8236
   macro avg       0.72      0.89      0.76      8236
weighted avg       0.93      0.87      0.88      8236



In [12]:
# Predict and evaluate
y_pred = rf.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.8637688198154444
[[6243 1065]
 [  57  871]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.85      0.92      7308
           1       0.45      0.94      0.61       928

    accuracy                           0.86      8236
   macro avg       0.72      0.90      0.76      8236
weighted avg       0.93      0.86      0.88      8236

